#CafChem tools for co-folding proteins and ligands using Boltz2.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/MauricioCafiero/CafChem/blob/main/notebooks/Boltz_CafChem.ipynb)

## This notebook allows you to:
- Input a protein sequence and a list of ligands to generate .yaml files for use in Boltz2.
- Run Boltz2 co-folding on the list of ligands in an automated fashion. Produces a list of IC50 values and binary binder/decoy scores.
- convert the .cif file to XYZ files for the complex, the protein and the ligand.
- visualize predicted data.

## Requirements:
- This notebook will install boltz2 and py3Dmol
- It will pull the CafChem tools from Github.
- It will install all needed libraries.
- small proteins can run on the L4 runtime, but larger ligands will need the memory provided by the A100 runtime.

### Install Boltz library
- will require an answer (y) and restart
- can take ~5 minutes

In [1]:
!pip uninstall torch torchvision

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Would remove:
    /usr/local/bin/torchfrtrace
    /usr/local/bin/torchrun
    /usr/local/lib/python3.11/dist-packages/functorch/*
    /usr/local/lib/python3.11/dist-packages/torch-2.6.0+cu124.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torch/*
    /usr/local/lib/python3.11/dist-packages/torchgen/*
Proceed (Y/n)? y
y
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Would remove:
    /usr/local/lib/python3.11/dist-packages/torchvision-0.21.0+cu124.dist-info/*
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libcudart.41118559.so.12
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libjpeg.1c1c4b09.so.8
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libnvjpeg.02b6d700.so.12
    /usr/local/lib/python3.11/dist-packages/torchvision.libs/libpng16.0364a1db.so.16
    /usr/local

In [2]:
! pip install torch torchvision torchaudio
! pip install py3Dmol
! pip install boltz -U

INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 131.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 102.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 64.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 41.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.2/158.2 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.6/216.6 

### Import libraries

In [2]:
!git clone https://github.com/MauricioCafiero/CafChem.git

Cloning into 'CafChem'...
remote: Enumerating objects: 107, done.
remote: Counting objects: 100% (107/107), done.
remote: Compressing objects: 100% (103/103), done.
remote: Total 107 (delta 52), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (107/107), 1.68 MiB | 8.13 MiB/s, done.
Resolving deltas: 100% (52/52), done.


In [3]:
import os
import CafChem.CafChemBoltz as ccb

## Co-fold ligands and proteins with Boltz2
- make_boltz_files tool accepts a list of SMILES, a protein sequence (single chain for now), the name of the protein, and an option list of ligand names.
  - returns .yaml files for each ligand with the protein, ready for analysis by Boltz.
- cofold tool accepts the names list returned by the previous tool and runs the analysis on each. returns a list of pIC50 values.

In [3]:
# sult1a3 example
mols = ["dop","ldop","para","napqi"]
smiles = ["NCCc1cc(O)c(O)cc1","C1=CC(=C(C=C1C[C@@H](C(=O)O)N)O)O","CC(=O)Nc1ccc(O)cc1","CC(=O)N=c1ccc(=O)cc1"]
seq = "MELIQDTSRPPLEYVKGVPLIKYFAEALGPLQSFQARPDDLLINTYPKSGTTWVSQILDMIYQGGDLEKCNRAPIYVRVPFLEVNDPGEPSGLETLKDTPPPRLIKSHLPLALLPQTLLDQKVKVVYVARNPKDVAVSYYHFHRMEKAHPEPGTWDSFLEKFMAGEVSYGSWYQHVQEWWELSRTHPVLYLFYEDMKENPKREIQKILEFVGRSLPEETMDFMVQHTSFKEMKKNPMTNYTTVPQELMDHSISPFMRKGMAGDWKTTFTVAQNERFDADYAEKMAGCSLSFRSEL"

In [4]:
# hmgcr example
mols = ["rosuvastatin"]
smiles = ["OC(=O)C[C@H](O)C[C@H](O)\C=C\c1c(C(C)C)nc(N(C)S(=O)(=O)C)nc1c2ccc(F)cc2"]
seq = "MLSRLFRMHGLFVASHPWEVIVGTVTLTICMMSMNMFTGNNKICGWNYECPKFEEDVLSS\
DIIILTITRCIAILYIYFQFQNLRQLGSKYILGIAGLFTIFSSFVFSTVVIHFLDKELTG\
LNEALPFFLLLIDLSRASTLAKFALSSNSQDEVRENIARGMAILGPTFTLDALVECLVIG\
VGTMSGVRQLEIMCCFGCMSVLANYFVFMTFFPACVSLVLELSRESREGRPIWQLSHFAR\
VLEEEENKPNPVTQRVKMIMSLGLVLVHAHSRWIADPSPQNSTADTSKVSLGLDENVSKR\
IEPSVSLWQFYLSKMISMDIEQVITLSLALLLAVKYIFFEQTETESTLSLKNPITSPVVT\
QKKVPDNCCRREPMLVRNNQKCDSVEEETGINRERKVEVIKPLVAETDTPNRATFVVGNS\
SLLDTSSVLVTQEPEIELPREPRPNEECLQILGNAEKGAKFLSDAEIIQLVNAKHIPAYK\
LETLMETHERGVSIRRQLLSKKLSEPSSLQYLPYRDYNYSLVMGACCENVIGYMPIPVGV\
AGPLCLDEKEFQVPMATTEGCLVASTNRGCRAIGLGGGASSRVLADGMTRGPVVRLPRAC\
DSAEVKAWLETSEGFAVIKEAFDSTSRFARLQKLHTSIAGRNLYIRFQSRSGDAMGMNMI\
SKGTEKALSKLHEYFPEMQILAVSGNYCTDKKPAAINWIEGRGKSVVCEAVIPAKVVREV\
LKTTTEAMIEVNINKNLVGSAMAGSIGGYNAHAANIVTAIYIACGQDAAQNVGSSNCITL\
MEASGPTNEDLYISCTMPSIEIGTVGGGTNLLPQQACLQMLGVQGACKDNPGENARQLAR\
IVCGTVMAGELSLMAALAAGHLVKSHMIHNRSKINLQDLQGACTKKTA"

In [6]:
files = ccb.make_boltz_files(smiles,seq,"HMGCR",mols)

In [7]:
pIC50s = ccb.cofold(files)

rosuvastatin done
pIC50 is: 7.274526964902878
IC50 is: 5.31463000484805e-08
Binder or Decoy: 0.46493685245513916


In [8]:
ccb.get_XYZ_files("rosuvastatin", "HMGCR")

In [9]:
f = open("rosuvastatin_HMGCR_ligand.xyz","r")
xyz = f.read()
f.close()

ccb.visualize_molecule(xyz)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.